<a href="https://colab.research.google.com/github/durgadevi0611/Python/blob/main/quickstarts/Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Prompting Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook contains examples of how to write and run your first prompts with the Gemini API.

In [ ]:
!pip install -U -q "google-generativeai>=0.7.2" # Install the Python SDK

In [ ]:
import google.generativeai as genai

## Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Run your first prompt

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to generate_content, and use the `.text` property to get the text content of the response.

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content("Give me python code to sort a list")
print(response.text)

```python
# There are several ways to sort a list in Python. Here are the most common:

# 1. Using the `sort()` method (in-place sorting):
def sort_list_in_place(my_list):
  """Sorts a list in place using the sort() method.  Modifies the original list."""
  my_list.sort()  # Sorts the list in ascending order by default.
  # To sort in descending order:  my_list.sort(reverse=True)
  return my_list  # Returns the *same* list, now sorted.  Not strictly necessary, but can be convenient.


# 2. Using the `sorted()` function (creates a new sorted list):
def sort_list_new(my_list):
  """Creates a new sorted list from the original list using the sorted() function.
     The original list remains unchanged."""
  new_list = sorted(my_list) # Returns a new sorted list, leaving the original untouched.
  # To sort in descending order:  new_list = sorted(my_list, reverse=True)
  return new_list


# 3. Sorting with a custom key (using `sort()` or `sorted()`):

def sort_list_by_length(my_list):
  """So

## Use images in your prompt

Here you will download an image from a URL and pass that image in our prompt.

First, you download the image and load it with PIL:

In [ ]:
!curl -o image.jpg "https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  2364k      0 --:--:-- --:--:-- --:--:-- 2375k


In [ ]:
import PIL.Image
img = PIL.Image.open('image.jpg')
img

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
prompt = """This image contains a sketch of a potential product along with some notes.
Given the product sketch, describe the product as thoroughly as possible based on what you
see in the image, making sure to note all of the product features. Return output in json format:
{description: description, features: [feature1, feature2, feature3, etc]}"""

Then you can include the image in our prompt by just passing a list of items to `generate_content`.

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content([prompt, img])
print(response.text)

```json
{
  "description": "The product is a jetpack backpack designed to look like a normal, lightweight backpack. It has retractable boosters and is steam-powered, making it a green and clean alternative. It is designed to fit an 18\" laptop.  The backpack is USB-C chargeable and has a 15-minute battery life. It also features padded strap support.",
  "features": [
    "Lightweight",
    "Looks like a normal backpack",
    "Fits 18\" laptop",
    "Retractable Boosters",
    "Steam-powered (Green/Clean)",
    "USB-C Charging",
    "15-minute battery life",
    "Padded Strap Support"
  ]
}
```


## Have a chat

The Gemini API enables you to have freeform conversations across multiple turns.

The [ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) class will store the conversation history for multi-turn interactions.

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')
chat = model.start_chat(history=[])

In [ ]:
response = chat.send_message("In one sentence, explain how a computer works to a young child.")
print(response.text)

A computer is like a super-smart toy that follows your instructions to play games, draw pictures, or tell you stories!



You can see the chat history:

In [ ]:
print(chat.history)

[parts {
  text: "In one sentence, explain how a computer works to a young child."
}
role: "user"
, parts {
  text: "A computer is like a super-smart toy that follows your instructions to play games, draw pictures, or tell you stories!\n"
}
role: "model"
]


You can keep sending messages to continue the conversation:

In [ ]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?")
print(response.text)

A computer operates by executing instructions stored in its memory, processing data through the CPU based on algorithms and logic, and utilizing input/output devices to interact with the user and external environment, all coordinated by an operating system that manages resources and provides a platform for applications.



## Set the temperature

Every prompt you send to the model includes parameters that control how the model generates responses. Use a `genai.GenerationConfig` to set these, or omit it to use the defaults.

Temperature controls the degree of randomness in token selection. Use higher values for more creative responses, and lower values for more deterministic responses.

You can set the `generation_config` when creating the model.

In [ ]:
model = genai.GenerativeModel(
    'gemini-2.0-flash',
    generation_config=genai.GenerationConfig(
        max_output_tokens=2000,
        temperature=0.9,
    ))

Or, set the `generation_config` on an individual call to `generate_content`. Any values set there override values on the model constructor.

Note: Although you can set the `candidate_count` in the generation_config, gemini-2.0-flash models will only return a single candidate at the this time.

In [ ]:
response = model.generate_content(
    'Give me a numbered list of cat facts.',
    # Limit to 5 facts.
    generation_config = genai.GenerationConfig(stop_sequences=['\n6'])
)

In [ ]:
print(response.text)

Okay, here's a numbered list of fascinating cat facts:

1.  **Cats have a third eyelid:** This is called a nictitating membrane, and it's a translucent eyelid that protects and moistens the eye. It's usually only visible when a cat is sleepy or sick.

2.  **A cat's nose print is unique:** Just like human fingerprints, no two cats have the same nose print pattern.

3.  **Cats can't taste sweetness:** They lack the taste receptor gene that allows mammals to taste sweet flavors.

4.  **Cats are crepuscular animals:** This means they are most active during dawn and dusk, which is when their prey (rodents) are also most active.

5.  **Cats have flexible bodies thanks to their "floating" clavicle:** Unlike humans, a cat's collarbone isn't attached to other bones, allowing for greater flexibility and range of motion.



## Learn more

There's lots more to learn!

* For more fun prompts, check out [Market a Jetpack](https://github.com/google-gemini/cookbook/blob/main/examples/Market_a_Jet_Backpack.ipynb).
* Check out the [safety quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Safety.ipynb) next to learn about the Gemini API's configurable safety settings, and what to do if your prompt is blocked.
* For lots more details on using the Python SDK, check out this [detailed quickstart](https://ai.google.dev/tutorials/python_quickstart).

In [3]:
s="Devi"
print(type(s))
print(len(s))
print(s.upper())

<class 'str'>
4
DEVI


In [10]:
d={"name":"devi","age":27}
print(type(d))
print(d["name"])
print(d["age"])

<class 'dict'>
devi
27


In [13]:
prices={"small":23,"medium":32,"big":22}
pepper={"small":3,"medium":2,"big":2}
size=input("tell me the size")
bill=prices[size]
if input("u want pepper:y/n")=='y':
  bill+=pepper[size]
if input("u want cheese:y/n")=="y":
  bill+=1
print("final bill:",bill)





tell me the sizesmall
u want pepper:y/ny
u want cheese:y/nn
final bill: 26


In [16]:
a=123
a=str(a)
print(type(a))

<class 'int'>


In [21]:
s="Alphabets"
print(s[:])
print(s[::])
print(s[1:]) #from index 1 to end of the string
print(s[:5]) #from starting index to 5(exclude)
print(s[2:6]) #from 2 to 5
print(s[1:6:2]) #from index 1 to 6 by stepping 2

Alphabets
Alphabets
lphabets
Alpha
phab
lhb


In [26]:
s="Alphabets"
print(s[-1:])
print(s[:-1])
print(s[-5:-1])
print(s[-8:-1:2])


s
Alphabet
abet
lhbt


In [1]:
#palindrome
s=input("enter a string")
if(s==s[::-1]):
  print("yes")
else:
  print("no")


enter a stringmam
yes


In [3]:
#strings are immutable
#concat
s="python"
s=s+"program"
print(s)


pythonprogram


In [4]:
#duplicate
s="*"
d=s*5
print(d)

*****


In [5]:
#membership operators(in, not in)
s="python"
if "p" in s:
  print("y")


y


In [7]:
#count vowels in the string
count=0
s="program"
for c in s:
  if c in "aeiouAEIOU":
    count=+1
print(count)


1


In [8]:
#case
s="python program"
print(s.upper())
print(s.lower())
print(s.title())
print(s.capitalize())
print(s.swapcase())

PYTHON PROGRAM
python program
Python Program
Python program
PYTHON PROGRAM
